In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = '../Data/Copy of Week2_challenge_data_source(CSV).csv'  # Update this with your dataset path
data = pd.read_csv(file_path)

# Inspect the dataset
print(data.head())
print(data.info())


In [ ]:
data.columns

In [ ]:
# Top 10 handsets
top_10_handsets = data['Handset Type'].value_counts().head(10)

# Visualization
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_handsets.values, y=top_10_handsets.index, palette="viridis")
plt.title('Top 10 Handsets Used by Customers')
plt.xlabel('Count')
plt.ylabel('Handset Type')
plt.show()


In [ ]:
# Top 3 manufacturers
top_3_manufacturers = data['Handset Manufacturer'].value_counts().head(3)

# Visualization
plt.figure(figsize=(8, 5))
sns.barplot(x=top_3_manufacturers.values, y=top_3_manufacturers.index, palette="mako")
plt.title('Top 3 Handset Manufacturers')
plt.xlabel('Count')
plt.ylabel('Manufacturer')
plt.show()


In [ ]:
# Top 3 manufacturers
top_3 = top_3_manufacturers.index

# Top 5 handsets per manufacturer
top_5_per_manufacturer = {}
for manufacturer in top_3:
    top_5_per_manufacturer[manufacturer] = data[data['Handset Manufacturer'] == manufacturer]['Handset Type'].value_counts().head(5)

# Visualization
plt.figure(figsize=(12, 8))
for manufacturer, handsets in top_5_per_manufacturer.items():
    sns.barplot(x=handsets.values, y=handsets.index, label=manufacturer)

plt.title('Top 5 Handsets per Top 3 Manufacturers')
plt.xlabel('Count')
plt.ylabel('Handset Type')
plt.legend(title="Manufacturer")
plt.show()


In [ ]:
# Aggregate per user
user_behavior = data.groupby('IMSI').agg({
    'Bearer Id': 'count',                 # Number of xDR sessions
    'Dur. (ms)': 'sum',                   # Total session duration
    'Total DL (Bytes)': 'sum',            # Total Download
    'Total UL (Bytes)': 'sum',            # Total Upload
    'Social Media DL (Bytes)': 'sum',     # Social Media Download
    'Social Media UL (Bytes)': 'sum',     # Social Media Upload
    'Youtube DL (Bytes)': 'sum',          # YouTube Download
    'Youtube UL (Bytes)': 'sum',          # YouTube Upload
    'Netflix DL (Bytes)': 'sum',          # Netflix Download
    'Netflix UL (Bytes)': 'sum',          # Netflix Upload
    'Google DL (Bytes)': 'sum',           # Google Download
    'Google UL (Bytes)': 'sum',           # Google Upload
    'Email DL (Bytes)': 'sum',            # Email Download
    'Email UL (Bytes)': 'sum',            # Email Upload
    'Gaming DL (Bytes)': 'sum',           # Gaming Download
    'Gaming UL (Bytes)': 'sum',           # Gaming Upload
    'Other DL (Bytes)': 'sum',            # Other Download
    'Other UL (Bytes)': 'sum'             # Other Upload
}).reset_index()

# Add total data volume
user_behavior['Total Data Volume (Bytes)'] = user_behavior['Total DL (Bytes)'] + user_behavior['Total UL (Bytes)']
print(user_behavior.head())


In [ ]:
# Summarize application usage
app_usage = user_behavior[['Social Media DL (Bytes)', 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 
                           'Google DL (Bytes)', 'Email DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']].sum()

# Visualization
plt.figure(figsize=(10, 6))
app_usage.plot(kind='pie', autopct='%1.1f%%', startangle=140, colormap='tab10')
plt.title('Distribution of Application Usage (Download Data)')
plt.ylabel('')
plt.show()


In [ ]:
# Scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=user_behavior, x='Total Data Volume (Bytes)', y='Dur. (ms)', alpha=0.7)
plt.title('Session Duration vs Total Data Volume')
plt.xlabel('Total Data Volume (Bytes)')
plt.ylabel('Session Duration (s)')
plt.xscale('log')
plt.yscale('log')
plt.show()



In [ ]:
# Data overview
print(data.info())
print(data.describe())

# Check for missing values
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)

# Treat missing values for numeric columns only
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

# Verify no missing values remain
print("Remaining Missing Values:\n", data.isnull().sum())






In [ ]:
# Function to replace outliers with the column mean
def replace_outliers_with_mean(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return column.apply(lambda x: column.mean() if x < lower_bound or x > upper_bound else x)

# Replace outliers for all numeric columns
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_columns] = data[numeric_columns].apply(replace_outliers_with_mean)

# Verify that no extreme outliers remain
print(data[numeric_columns].describe())


In [ ]:
# Calculate total duration per user
data['Total Duration (s)'] = data['Dur. (ms)'] / 1000  # Convert ms to seconds

# Segment into decile classes, dropping duplicate edges
data['Decile Class'] = pd.qcut(data['Total Duration (s)'], 10, labels=range(1, 11), duplicates="drop")

# Compute total data (DL + UL) per decile
decile_summary = data.groupby('Decile Class').agg({
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum'
})
decile_summary['Total Data (Bytes)'] = decile_summary['Total DL (Bytes)'] + decile_summary['Total UL (Bytes)']

print(decile_summary)

# Ensure all necessary columns are present and numeric
numeric_columns = ['Dur. (ms)', 'Total DL (Bytes)', 'Total UL (Bytes)']
data[numeric_columns] = data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Calculate total duration per user
data['Total Duration (s)'] = data['Dur. (ms)'] / 1000  # Convert ms to seconds

# Segment into decile classes, dropping duplicate edges
data['Decile Class'] = pd.qcut(data['Total Duration (s)'], 10, labels=range(1, 11), duplicates="drop")

# Compute total data (DL + UL) per decile
decile_summary = data.groupby('Decile Class').agg({
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum'
})
decile_summary['Total Data (Bytes)'] = decile_summary['Total DL (Bytes)'] + decile_summary['Total UL (Bytes)']

print(decile_summary)



In [ ]:
# Basic metrics for quantitative variables
metrics = data[numeric_columns].agg(['mean', 'median', 'std', 'min', 'max']).T
print(metrics)


In [ ]:
# Relationship between applications and total data
apps = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)',
        'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']

for app in apps:
    plt.figure(figsize=(8, 5))
    sns.scatterplot(data=data, x=app, y='Total DL (Bytes)')
    plt.title(f'{app} vs Total DL Data')
    plt.xlabel(app)
    plt.ylabel('Total DL Data')
    plt.show()



In [ ]:
# Correlation matrix
selected_apps = ['Social Media DL (Bytes)', 'Google DL (Bytes)', 'Email DL (Bytes)',
                 'Youtube DL (Bytes)', 'Netflix DL (Bytes)', 'Gaming DL (Bytes)', 'Other DL (Bytes)']
correlation_matrix = data[selected_apps].corr()

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix')
plt.show()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Standardize the data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data[selected_apps])

# Perform PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(data_scaled)

# Explained variance
explained_variance = pca.explained_variance_ratio_
print("Explained Variance Ratios:", explained_variance)
